[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Python-Financial-Analyst/pyfian_dev/blob/main/notebooks/fixed_)


In [8]:
from pyfian.time_value.irr import irr

irr([-1000, 300, 420, 680, 900], guess=0.1)

from pyfian.time_value.future_value import future_value_annuity
future_value_annuity([-1000, 300, 420, 680, 900], rate=0.1, periods=5)



TypeError: can't multiply sequence by non-int of type 'float'